In [2]:
print ("okey")

okey


In [3]:
%pwd

'c:\\Users\\md.ashraf\\Desktop\\KTC-AI\\KTC-ACC-Generative-AI\\KTC-ACC-Generative-AI\\research'

In [34]:
import os
os.chdir("../")

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data from the PDF File

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [ ]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
extracted_data

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2023-01-13T15:13:02+08:00', 'moddate': '2023-01-13T15:13:02+08:00', 'source': 'Data\\16a_CR206_GS App P1_Tunnel Machine.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}, page_content='Page i of 35 \nGENERAL SPECIFICATION \n \nAPPENDIX P1 \n \nTUNNEL MACHINES'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2023-01-13T15:13:02+08:00', 'moddate': '2023-01-13T15:13:02+08:00', 'source': 'Data\\16a_CR206_GS App P1_Tunnel Machine.pdf', 'total_pages': 35, 'page': 1, 'page_label': '2'}, page_content='GS-P1-i \n \n \nGENERAL SPECIFICATION \n \nAPPENDIX P \n \nTUNNEL MACHINES \n \n CONTRACT CR206 \nClause No  Title            Page GS-P1- \n1. SCOPE ............................................................................................................ 1 \n2. DEFINITIONS .................................................................................................. 2 \n3. SEL

In [9]:
#Split the data into text chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks" , len(text_chunks))

Length of Text Chunks 177


In [11]:
# pip install -U sentence-transformers

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\md.ashraf\AppData\Local\Temp\ipykernel_50836\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\md.ashraf\AppData\Local\anaconda3\envs\ktcbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\md.ashraf\AppData\Local\anaconda3\envs\ktcbot\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default 

In [15]:
query_result = embeddings.embed_query("Hello world")
print ("Length" , len(query_result))

Length 384


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [19]:
# Import the Pinecone library
from pinecone import Pinecone
from pinecone import ServerlessSpec

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ktc-bot"

# Create the index
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'X-Cloud-Trace-Context': 'ccb56543f27e4176b18f2631998ee88f', 'Date': 'Tue, 18 Mar 2025 06:53:27 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [20]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [22]:
#Load Existing Index

from langchain_pinecone import PineconeVectorStore

#Embed Each Chunk and upsert the embeddings into your pinecone index

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embeddings,
)

In [23]:
docsearch

In [24]:
retriever = docsearch.as_retriever(search_type="similarity" , search_kwargs={"k" :3})

In [25]:
retrieved_docs = retriever.invoke("What is TBM?")

In [26]:
retrieved_docs

[Document(id='77380cc7-903a-4492-9807-c042d359f52c', metadata={'creationdate': '2023-01-13T15:13:02+08:00', 'creator': 'PyPDF', 'moddate': '2023-01-13T15:13:02+08:00', 'page': 3.0, 'page_label': '4', 'producer': 'PyPDF', 'source': 'Data\\16a_CR206_GS App P1_Tunnel Machine.pdf', 'total_pages': 35.0}, page_content='GS-P1-2 \n CONTRACT CR206 \n2. DEFINITIONS \n2.1. “TBM” (Tunnel Boring Machine) is a machine equipped with excavating \nmachinery that is i ntegral with the shield and which excavates the full \ntunnel profile. \nTBM is a type of pressure balanced machine equipped with a separate \nchamber behind the cutter head where a mixture of earth and water \n(Earth Pressure Balance Machine – EPB TBM), slurry (S lurry TBM) or \na similar medium is pressurised in the excavation chamber to balance'),
 Document(id='4135e932-dcbe-4737-8923-958b5310e07c', metadata={'creationdate': '2023-01-13T15:13:02+08:00', 'creator': 'PyPDF', 'moddate': '2023-01-13T15:13:02+08:00', 'page': 3.0, 'page_label

In [27]:
from langchain_openai import OpenAI
llm = OpenAI(temperature = 0.4 , max_tokens= 500)

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Define system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise and precise.\n\n"
    "{context}"
)

# Create a ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])



In [31]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [32]:
response = rag_chain.invoke({"input" : "What is TBM?"})
print(response["answer"])



TBM stands for Tunnel Boring Machine and it is a type of machine that excavates the full tunnel profile. It is equipped with excavating machinery that is integral with the shield and has a separate chamber behind the cutter head where a mixture of earth and water, slurry, or a similar medium is pressurized to balance the excavation.


In [33]:
response = rag_chain.invoke({"input" : "What are the colour coded for TBM?"})
print(response["answer"])



The services and utilities on site and on the TBM are to be colour coded to prevent accidental cross contamination and to warn where lines are located. This information should be provided 1 week prior to a TBM shipment to Singapore.
